In [1]:
import pandas as pd
import numpy as np
import json
import ast
import os

In [2]:
from tqdm.notebook import tqdm

In [3]:
tqdm.pandas()

In [4]:

args = pd.Series({
    "root_dir":"/mnt/disks/data/",
    "dataset_path":"/mnt/disks/data/fma/fma_large",
    "embeddings":"music_style",
    "train_id": "hierarchical_single",
    'sample_size': 1
})


In [5]:

job_path = "/mnt/disks/data/fma/trains"

# In[15]:

train_path = os.path.join(job_path,args.train_id)

# In[16]:

base_path = os.path.join(args.root_dir,"fma")

# In[17]:

models_path = os.path.join(args.root_dir,"models")


metadata_path_fma = os.path.join(base_path,"fma_metadata")

# In[18]:

metadata_file = os.path.join(train_path,"metadata.json")


categories_labels_path = os.path.join(train_path,"labels.json")


In [6]:


def __load_json__(path):
    with open(path, 'r') as f:
        tmp = json.loads(f.read())

    return tmp




In [7]:

def create_dir(path):
    # checking if the directory demo_folder2 
    # exist or not.
    if not os.path.isdir(path):

        # if the demo_folder2 directory is 
        # not present then create it.
        os.makedirs(path)
    return True



In [8]:
create_dir(train_path)

True

## Load genres file. Contains relationships beetwen genres

In [9]:
genres_df = pd.read_csv(os.path.join(metadata_path_fma,'genres.csv'))


In [10]:
genres_df

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
...,...,...,...,...,...
158,1032,60,102,Turkish,2
159,1060,30,46,Tango,2
160,1156,26,130,Fado,2
161,1193,72,763,Christmas,38


In [11]:
genres_df[genres_df['genre_id'] == 495]

,genre_id,#tracks,parent,title,top_level
136,495,2061,15,Downtempo,15


In [12]:
# Cria um dicionário que associa o ID de cada música aos IDs de seus gêneros musicais
tracks_df = pd.read_csv(os.path.join(metadata_path_fma,'tracks_valid.csv'))

In [13]:
tracks_df = tracks_df.sample(frac=args.sample_size)

In [14]:
tracks_df.sample(20)

,track_id,track_title,valid_genre
9273,15752,Channeling,"['1', '38', '74']"
28218,43592,Am Pistol,['524']
101526,151019,凌ぎ場,"['15', '21']"
74666,115114,"Oh, You Touched It.",['76']
92795,138917,Sulla Costruzione Del Ponte Sull'Adda,['137']
69157,107895,Fashion Statement,"['17', '25', '53']"
42566,64447,Daisy Beater,"['42', '495']"
34979,53886,Все как один,"['25', '38']"
14081,23086,les petits ananas de Carlos,"['15', '76', '183']"
45951,68996,Romane,"['15', '100', '296']"


In [15]:
tracks_df.valid_genre.values

array(["['1', '38', '250']", "['17', '33', '180']",
       "['15', '32', '38', '41']", ..., "['15', '38', '107']",
       "['10', '15']", "['15', '32', '38', '41', '1235']"], dtype=object)

In [16]:
tracks_df.track_title

19671       Tetsuki Continuum
16511     The Divine Invasion
98019                  Phyral
89391    Drum Circle, Gokarna
52886               Synthro 2
                 ...         
61619                O.H. Lee
44076                FF Nexus
90320                       k
31434                Republic
96988            17112012-000
Name: track_title, Length: 104186, dtype: object

In [17]:
## Get complete genre structure
def get_all_structure(estrutura,df_genres):
    ## Get structure from df_genres
    def get_all_structure_from_df(estrutura,df_genres,structure=[]):
        if estrutura == 0:
            return structure
        else:
            structure.append(int(estrutura))
            get_all_structure_from_df(df_genres[df_genres["genre_id"]==int(estrutura)].parent.values[0],df_genres,structure)
            return structure
    
    return get_all_structure_from_df(estrutura,df_genres,structure=[])
    

In [18]:
# tracks_df['valid_genre'] = tracks_df.track_genres.apply(lambda x: x.strip('][').split(', ') if x != '[]' else None)
tracks_df['valid_genre'] = tracks_df.valid_genre.apply(lambda x: ast.literal_eval(x))

In [19]:
tracks_df['last_genre_id'] = tracks_df.valid_genre.apply(lambda x:x[-1] if x != None else None)

In [20]:
tracks_df.sample(20)

,track_id,track_title,valid_genre,last_genre_id
68408,106972,MTE,[236],236
103182,154131,Pitch Control,"[15, 32, 38, 41]",41
48413,72403,american (tongue vision),"[15, 38]",38
9570,16162,Rock da Dam (Featuring Portformat),[21],21
75439,116087,Waking Up (Remix),"[15, 76, 1235]",1235
6063,11347,Cantate Domino,[5],5
103391,154355,Poles Apart,[296],296
57483,86450,Taste thiz shit,"[183, 240]",240
12178,20160,Traviata Sempre Libera,[8],8
72635,112485,Erykah & Jean (Instrumental),[21],21


In [21]:
tracks_df.dropna(inplace=True)

In [22]:
tracks_df

,track_id,track_title,valid_genre,last_genre_id
19671,31829,Tetsuki Continuum,"[1, 38, 250]",250
16511,27258,The Divine Invasion,"[17, 33, 180]",180
98019,146281,Phyral,"[15, 32, 38, 41]",41
89391,134295,"Drum Circle, Gokarna","[30, 86, 224]",224
52886,80657,Synthro 2,"[38, 580]",580
...,...,...,...,...
61619,94550,O.H. Lee,"[12, 25, 58]",58
44076,66264,FF Nexus,"[76, 224]",224
90320,135824,k,"[15, 38, 107]",107
31434,48612,Republic,"[10, 15]",15


In [23]:
tracks_df['full_genre_id'] = tracks_df.last_genre_id.progress_apply(lambda x: get_all_structure(x,genres_df)[::-1])

  0%|          | 0/104185 [00:00<?, ?it/s]

In [24]:
tracks_df.full_genre_id.value_counts()

[38]             6508
[1235]           6043
[10, 76]         4126
[1235, 107]      3988
[17, 103]        3482
                 ... 
[20, 65, 189]       4
[9, 651, 493]       4
[2, 86, 173]        4
[20, 7]             2
[5, 444]            2
Name: full_genre_id, Length: 159, dtype: int64

In [25]:
tracks_df.columns

Index(['track_id', 'track_title', 'valid_genre', 'last_genre_id',
       'full_genre_id'],
      dtype='object')

In [26]:
tracks_df = tracks_df[['track_id','full_genre_id']]

In [27]:
tracks_df.full_genre_id.values

array([list([38, 250]), list([17, 180]), list([38, 41]), ...,
       list([1235, 107]), list([15]), list([1235])], dtype=object)

In [28]:
tracks_df.full_genre_id.info

<bound method Series.info of 19671      [38, 250]
16511      [17, 180]
98019       [38, 41]
89391      [38, 224]
52886      [21, 580]
            ...     
61619       [12, 58]
44076      [38, 224]
90320    [1235, 107]
31434           [15]
96988         [1235]
Name: full_genre_id, Length: 104185, dtype: object>

In [29]:
labels_size = tracks_df.full_genre_id.apply(lambda x: len(x))

In [30]:
labels_size = labels_size.max()

In [31]:
labels_size

5

In [32]:
import pandas as pd
import os

### Parse of label to structure

In [33]:
### Function for parse label to sctructure of hierarhical scheme

def parse_label(label,label_size=5):
    # label = label.split('-')
    # preencher com 0 no caso de haver menos de 5 níveis
    labels = np.zeros(label_size,dtype=int)
    for i, label in enumerate(label):
        if i == 5:
            break
        # Aqui você pode fazer a conversão do label em um índice inteiro usando um dicionário ou outro método
        # Neste exemplo, estou apenas usando a posição da label na lista como índice
        labels[i] = label
    return labels

In [34]:
parsed_labels = tracks_df.full_genre_id.apply(lambda x: parse_label(x))

In [35]:
tracks_df['full_genre_id']

19671      [38, 250]
16511      [17, 180]
98019       [38, 41]
89391      [38, 224]
52886      [21, 580]
            ...     
61619       [12, 58]
44076      [38, 224]
90320    [1235, 107]
31434           [15]
96988         [1235]
Name: full_genre_id, Length: 104185, dtype: object

In [36]:
def convert_label_to_string(x,level=2):
    return '-'.join([str(value) for value in x[:level]])

In [37]:
tracks_df['labels_1'] = parsed_labels.progress_apply(lambda x: str(x[:1][0]))
tracks_df['labels_2'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=2))
tracks_df['labels_3'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=3))
tracks_df['labels_4'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=4))
tracks_df['labels_5'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=5))

  0%|          | 0/104185 [00:00<?, ?it/s]

/tmp/ipykernel_211846/310633048.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df['labels_1'] = parsed_labels.progress_apply(lambda x: str(x[:1][0]))


  0%|          | 0/104185 [00:00<?, ?it/s]

/tmp/ipykernel_211846/310633048.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df['labels_2'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=2))


  0%|          | 0/104185 [00:00<?, ?it/s]

  0%|          | 0/104185 [00:00<?, ?it/s]

  0%|          | 0/104185 [00:00<?, ?it/s]

In [38]:
tracks_df.labels_1.value_counts()

38      22066
12      21710
15      18492
1235    11214
17       6900
10       6514
21       6453
2        3331
5        2456
20       1183
4        1147
9        1023
8         620
13        466
14        430
3         180
Name: labels_1, dtype: int64

In [39]:
tracks_df['labels_1'].value_counts()

38      22066
12      21710
15      18492
1235    11214
17       6900
10       6514
21       6453
2        3331
5        2456
20       1183
4        1147
9        1023
8         620
13        466
14        430
3         180
Name: labels_1, dtype: int64

In [40]:
tracks_df = tracks_df[tracks_df['labels_1'].isin(['3','14'])]

In [41]:
tracks_df.to_csv(os.path.join(train_path,"tracks.csv"),index=False)

In [42]:
tracks_df = pd.read_csv(os.path.join(train_path,"tracks.csv"))

In [43]:
categories_df = pd.DataFrame({'level5':tracks_df.labels_5.unique()})

In [44]:
categories_df['level1'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:1]))
categories_df['level2'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:2]))
categories_df['level3'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:3]))
categories_df['level4'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:4]))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [45]:
categories_df

,level5,level1,level2,level3,level4
0,3-0-0-0-0,3,3-0,3-0-0,3-0-0-0
1,14-0-0-0-0,14,14-0,14-0-0,14-0-0-0
2,14-11-0-0-0,14,14-11,14-11-0,14-11-0-0
3,14-19-0-0-0,14,14-19,14-19-0,14-19-0-0
4,3-567-0-0-0,3,3-567,3-567-0,3-567-0-0


In [46]:
def get_labels_name(x,genres_df):
    levels = 5
    full_name = []
    last_level = 0
    genre_root = ""
    for genre in x.split('-'):
        genre_df = genres_df[genres_df['genre_id'] == int(genre)]
        if genre_df.empty:
            genre_name = genre_root 
        else:
            genre_name = genre_df.title.values.tolist()[0]
            genre_root = genre_name
        
        full_name.append(genre_name)
    full_name = '>'.join(full_name)
        
    return full_name
    # return genres_df[genres_df['genre_id'] == int(x)].title.values.tolist()[0]

In [47]:
categories_df['level5_name'] = categories_df.level5.apply(lambda x: get_labels_name(x,genres_df))

In [48]:
categories_df

,level5,level1,level2,level3,level4,level5_name
0,3-0-0-0-0,3,3-0,3-0-0,3-0-0-0,Blues>Blues>Blues>Blues>Blues
1,14-0-0-0-0,14,14-0,14-0-0,14-0-0-0,Soul-RnB>Soul-RnB>Soul-RnB>Soul-RnB>Soul-RnB
2,14-11-0-0-0,14,14-11,14-11-0,14-11-0-0,Soul-RnB>Disco>Disco>Disco>Disco
3,14-19-0-0-0,14,14-19,14-19-0,14-19-0-0,Soul-RnB>Funk>Funk>Funk>Funk
4,3-567-0-0-0,3,3-567,3-567-0,3-567-0-0,Blues>Gospel>Gospel>Gospel>Gospel


In [49]:
def __create_labels__(categories_df):
    data = {
        "label1": {},
        "label2": {},
        "label3": {},
        "label4": {},
        "label5": {},
        "label1_inverse": [],
        "label2_inverse": [],
        "label3_inverse": [],
        "label4_inverse": [],
        "label5_inverse": [],
        "label1_name": {},
        "label2_name": {},
        "label3_name": {},
        "label4_name": {},
        "label5_name": {},
    }

    idx = 0
    
    for id_x, cat in enumerate(set(categories_df.level1.values.tolist())):
        data['label1'][cat] = idx
        data['label1_inverse'].append(cat)
        data['label1_count'] = idx + 1
        idx+=1

    for id_x, cat in enumerate(set(categories_df.level2.values.tolist())):
        data['label2'][cat] = idx
        data['label2_inverse'].append(cat)
        data['label2_count'] = idx + 1
        idx+=1
        
    for id_x, cat in enumerate(set(categories_df.level3.values.tolist())):
        data['label3'][cat] = idx
        data['label3_inverse'].append(cat)
        data['label3_count'] = idx + 1
        idx+=1

    for id_x, cat in enumerate(set(categories_df.level4.values.tolist())):
        data['label4'][cat] = idx
        data['label4_inverse'].append(cat)
        data['label4_count'] = idx + 1
        idx+=1
        
    for idx, cat in enumerate(set(categories_df.level5.values.tolist())):
        data['label5'][cat] = idx
        data['label5_inverse'].append(cat)
        data['label5_count'] = idx + 1
        idx+=1
        
    for cat5,cat1,cat2,cat3,cat4,name5 in categories_df.values:
        
        name1 = '>'.join(name5.split('>')[:1])
        name2 = '>'.join(name5.split('>')[:2])
        name3 = '>'.join(name5.split('>')[:3])
        name4 = '>'.join(name5.split('>')[:4])
        
        
        data['label1_name'][cat1] = name1
        data['label2_name'][cat2] = name2
        data['label3_name'][cat3] = name3
        data['label4_name'][cat4] = name4
        data['label5_name'][cat5] = name5
        
    return data

In [50]:
with open(categories_labels_path, 'w+') as f:
    f.write(json.dumps(__create_labels__(categories_df)))

In [51]:
labels  = __create_labels__(categories_df)

In [52]:
labels['label4']

{'14-11-0-0': 12,
 '3-567-0-0': 13,
 '3-0-0-0': 14,
 '14-19-0-0': 15,
 '14-0-0-0': 16}